In [1]:
import os
import datetime
import copy
import re
import yaml
import uuid
import warnings
import time
import inspect

import numpy as np
import pandas as pd
from functools import partial, reduce
from random import shuffle
import random

import torch
from torch import nn, optim
from torch import nn
from torch.nn import functional as F
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader
from torchvision.models import resnet
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision.datasets import MNIST
import tensorflow as tf
from tqdm.autonotebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn import metrics as mtx
from sklearn import model_selection as ms

#### 수정 전

In [2]:
def get_data_loaders(train_batch_size, val_batch_size):
    mnist = MNIST(download=True, train=True, root=".").train_data.float()
    print(mnist.shape, len(mnist))
    data_transform = Compose([ Resize((224, 224)),ToTensor(), Normalize((mnist.mean()/255,), (mnist.std()/255,))])

    train_loader = DataLoader(MNIST(download=True, root=".", transform=data_transform, train=True),
                              batch_size=train_batch_size, shuffle=True)
    val_loader = DataLoader(MNIST(download=False, root=".", transform=data_transform, train=False),
                            batch_size=val_batch_size, shuffle=False)
    return train_loader, val_loader

In [3]:
train_batch_size = 256
val_batch_size = 256

train_loader, valid_loader = get_data_loaders(train_batch_size, val_batch_size)

/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:62: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


torch.Size([60000, 28, 28]) 60000


In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [5]:
x_train = x_train[:30001]
y_train = y_train[:30001]
x_test = x_test[:9000]
y_test = y_test[:9000]

In [6]:
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])

x_train shape: (30001, 28, 28)
Number of images in x_train 30001
Number of images in x_test 9000


In [7]:
instance_index_label = [(i, y_train[i]) for i in range(x_train.shape[0])]
instance_index_label_test = [(i, y_test[i]) for i in range(x_test.shape[0])]

In [33]:
train_batch_size = 1
val_batch_size = 1
train_loader, val_loader = get_data_loaders(train_batch_size, val_batch_size)
loss_function = nn.CrossEntropyLoss() # your loss function, cross entropy works well for multi-class problems

# optimizer
optimizer = optim.Adadelta(model.parameters())

torch.Size([60000, 28, 28]) 60000


In [44]:
28*28

784

In [34]:
losses = []
batches = len(train_loader)
val_batches = len(valid_loader)

In [9]:
import torch
from torchvision.models.resnet import ResNet, BasicBlock
class MnistResNet(ResNet):
    def __init__(self):
        super(MnistResNet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10)
        self.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
    def forward(self, x):
        return torch.softmax(super(MnistResNet, self).forward(x), dim=-1)

In [10]:
model = MnistResNet()
# model.load_state_dict(torch.load('mnist_state.pt'))
body = nn.Sequential(*list(model.children()))
# extract the last layer
model = body[:9]
# the model we will use
model.eval()

Sequential(
  (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [35]:
# loop for every epoch (training + evaluation)
meta_table = dict()
feature_result = []

# progress bar
progress = tqdm(enumerate(train_loader), desc="Loss: ", total=batches)

model.eval()

for i, data in progress:
    if i==30001:
        break
    X, y = data[0], data[1]
    # training step for single batch
    model.zero_grad()
    outputs = model(X)
    feature_result.append(outputs.reshape(-1).tolist())
    meta_table[i] = outputs.reshape(-1).tolist()
    
feature_array = np.array(feature_result)
np.save('feature_array_full',feature_array )

Loss:   0%|          | 0/60000 [00:00<?, ?it/s]

In [45]:
feature_array

array([[1.79757905, 1.0747999 , 0.64502156, ..., 0.05546547, 3.39392066,
        2.90366793],
       [1.79465747, 1.11855567, 0.78918833, ..., 0.06835514, 3.10944915,
        2.80919099],
       [1.14950323, 0.92153323, 0.38944232, ..., 0.05407333, 2.30370116,
        2.14239812],
       ...,
       [1.59912229, 1.05197775, 0.80092978, ..., 0.05676577, 3.25290704,
        2.69371104],
       [1.13229823, 0.66984236, 0.44521028, ..., 0.0729371 , 2.10821939,
        1.64437366],
       [1.33589578, 1.04693711, 0.58522958, ..., 0.05024722, 2.75830173,
        2.10698771]])

In [46]:
feature_array[0]

array([1.79757905e+00, 1.07479990e+00, 6.45021558e-01, 1.09722173e+00,
       4.72716856e+00, 1.36111248e+00, 4.58656502e+00, 3.89535695e-01,
       1.37735593e+00, 3.92084837e+00, 4.61730242e+00, 4.97283898e-02,
       3.64919633e-01, 1.80825281e+00, 2.84661484e+00, 1.73161253e-02,
       9.80857670e-01, 5.36403370e+00, 3.87409377e+00, 2.65567541e+00,
       2.04561210e+00, 4.06011057e+00, 2.76959252e+00, 1.57946408e+00,
       6.92420244e-01, 3.74062747e-01, 0.00000000e+00, 2.37332535e+00,
       2.34679922e-01, 1.25870323e+00, 1.40805221e+00, 2.67255449e+00,
       1.06955099e+00, 7.13593811e-02, 1.99983966e+00, 2.10215092e+00,
       1.70896208e+00, 1.13451079e-01, 7.89465070e-01, 1.66233432e+00,
       2.44982290e+00, 2.15136480e+00, 2.83075362e-01, 2.06879449e+00,
       1.96311748e+00, 9.02110279e-01, 4.96351277e-04, 1.34481215e+00,
       1.09923422e+00, 5.14763832e+00, 3.42703438e+00, 1.28718764e-01,
       1.22739196e+00, 4.01055670e+00, 4.89475870e+00, 1.77836084e+00,
      

In [37]:
len(feature_array[0]) 

512

In [38]:
type(feature_array), feature_array.shape

(numpy.ndarray, (30001, 512))

In [39]:
len(instance_index_label)

30001

In [40]:
def data_generation(instance_index_label):
    """
    bags: {key1: [ind1, ind2, ind3],
           key2: [ind1, ind2, ind3, ind4, ind5],
           ... }
    bag_lbls:
        {key1: 0,
         key2: 1,
         ... }
    """
    bag_size = np.random.randint(3,7,size=len(instance_index_label)//5)
    data_cp = copy.copy(instance_index_label)
    np.random.shuffle(data_cp)
    bags = {}
    bags_per_instance_labels = {}
    bags_labels = {}
    for bag_ind, size in enumerate(bag_size):
        bags[bag_ind] = []
        bags_per_instance_labels[bag_ind] = []
        try:
            for _ in range(size):
                inst_ind, lbl = data_cp.pop()
                bags[bag_ind].append(inst_ind)
                # simplfy, just use a temporary variable instead of bags_per_instance_labels
                bags_per_instance_labels[bag_ind].append(lbl)
            bags_labels[bag_ind] = bag_label_from_instance_labels(bags_per_instance_labels[bag_ind])
        except:
            break
    return bags, bags_labels

def bag_label_from_instance_labels(instance_labels):
    return int(any(((x==1) for x in instance_labels)))

In [41]:
bag_indices, bag_labels = data_generation(instance_index_label)
bag_features = {kk: torch.Tensor(feature_array[inds]) for kk, inds in bag_indices.items()}

In [42]:
len(bag_indices), len(bag_labels)

(6000, 6000)

In [43]:
bag_indices.items()

dict_items([(0, [417, 10058, 14341, 23460, 27315, 16917]), (1, [25056, 24640, 9066, 11585, 25730]), (2, [27694, 24851, 19492, 19064]), (3, [27591, 7045, 249]), (4, [5699, 22095, 16446, 14184, 1889, 28435]), (5, [14604, 9584, 12214, 18869, 4560, 4168]), (6, [2616, 12485, 18449, 29665, 13063]), (7, [24465, 29907, 5227, 28742, 24554, 19108]), (8, [20559, 22833, 6687, 26912]), (9, [5481, 28264, 7141, 9719]), (10, [29239, 5009, 17589, 8735]), (11, [15686, 18179, 13680, 21994, 16527]), (12, [11543, 2834, 23562, 22650, 22778]), (13, [2419, 16741, 16700, 2951]), (14, [18579, 28785, 23480]), (15, [11661, 5412, 4357]), (16, [27793, 19962, 18751, 12805, 2327, 11242]), (17, [7398, 11825, 1432, 25392, 23522, 15253]), (18, [5952, 6697, 9055, 227, 27940]), (19, [17878, 5897, 21415, 6000, 14154]), (20, [10425, 21440, 18235]), (21, [10033, 13788, 18139]), (22, [20822, 7858, 24045, 8190, 14303]), (23, [1368, 6928, 23804, 19994, 15037]), (24, [19109, 1404, 22246]), (25, [22581, 185, 5613, 25048]), (26, [